<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/CakmaustatRSIBTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tvDatafeed pandas numpy matplotlib ta tradingview-screener

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def calculate_rsi(df, period=66):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def detect_rsi_break(df):
    rsi = calculate_rsi(df)
    df['RSI'] = rsi

    pivots = {'low': [], 'high': []}

    for i in range(1, len(rsi)-1):
        if rsi.iloc[i-1] > rsi.iloc[i] < rsi.iloc[i+1]:
            pivots['low'].append((i, rsi.iloc[i]))
        if rsi.iloc[i-1] < rsi.iloc[i] > rsi.iloc[i+1]:
            pivots['high'].append((i, rsi.iloc[i]))

    if not pivots['low'] or not pivots['high']:
        return False, rsi.iloc[-1]

    for low in pivots['low']:
        for high in pivots['high']:
            if low[0] > high[0] and low[1] < high[1]:
                if rsi.iloc[-1] > high[1]:
                    return True, rsi.iloc[-1]

    return False, rsi.iloc[-1]

def get_interval(period):
    intervals = {
        '30min': Interval.in_30_minute,
        '45min': Interval.in_45_minute,
        '2h': Interval.in_2_hour,
        '4h': Interval.in_4_hour,
        '1day': Interval.in_daily
    }
    return intervals.get(period, Interval.in_daily)  # Varsayılan olarak günlük

def main():
    tv = TvDatafeed()

    # Kullanıcıdan dönem ve borsa seçimini al
    period = input("Lütfen dönem seçimini yapın (30min, 45min, 2h, 4h, 1day): ").strip().lower()
    interval = get_interval(period)

    exchange = input("Lütfen borsa seçimini yapın (BINANCE, MEXC): ").strip().upper()
    if exchange not in ['BINANCE', 'MEXC']:
        print("Geçersiz borsa seçimi. Lütfen 'BINANCE' veya 'MEXC' girin.")
        return

    try:
        # Kripto sembollerini çek
        cryptos = get_all_symbols(market='crypto')

        # USDT paritelerini filtrele
        usdt_pairs = [symbol for symbol in cryptos if symbol.endswith('USDT') and exchange in symbol] #filtrelemek istediğiniz bir uzantı varsa buradan yapılabilir...sadece tırnak içi değişecek.

        usdt_pairs = sorted(usdt_pairs)
    except Exception as e:
        print(f"Kripto paraları çekerken hata alındı: {e}")
        return

    for crypto in usdt_pairs:
        try:
            df = tv.get_hist(crypto, exchange, interval)
            if df is None or df.empty:
                continue
            detected, rsi_value = detect_rsi_break(df)
            if detected and rsi_value > 30:  # rsi 30 uzerındeki kırılımları alıyor.. bu değeride değiştirebilirsiniz...
                print(f"{crypto} için RSI düşeni kırdı! RSI Değeri: {rsi_value:.2f}")
        except Exception as e:
            print(f"{crypto} için veri işleme hatası: {e}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Program kesildi.")
#tarama ortalama 5 - 7 dk arası sürüyor... tamamlandıktan sonra rsi değeri büyükten küçüğe doğru tekrardan filtreleyecek period seçimi sonrası taramayı başlatıp bekleyeceksiniz.. kolay gelsin.

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-3z2bvoh_
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-3z2bvoh_
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Lütfen dönem seçimini yapın (30min, 45min, 2h, 4h, 1day): 1day
Lütfen borsa seçimini yapın (BINANCE, MEXC): BINANCE
BINANCE:AEURUSDT için RSI düşeni kırdı! RSI Değeri: 76.68
BINANCE:BAKEUSDT için RSI düşeni kırdı! RSI Değeri: 74.24
BINANCE:BONKUSDT için RSI düşeni kırdı! RSI Değeri: 30.83
BINANCE:EURUSDT için RSI düşeni kırdı! RSI Değeri: 75.65
BINANCE:FTTUSDT için RSI düşeni kırdı! RSI Değeri: 32.20
BINANCE:HARDUSDT için RSI düşeni kırdı! RSI Değeri: 37.68
BINANCE:JUPUSDT için RSI düşeni kırdı! RSI Değeri: 34.91
BINANCE:NOTUSDT için RSI düşeni kırdı! RSI Değeri: 44.44
BINANCE:SFPUSDT için RSI düşeni kırdı! RSI Değeri: 69.23
BINANCE:TUSDUSDT için RSI düşeni kırdı! RSI Değeri: 31.37
BINANCE:WRXUSDT için RSI düşeni kırdı! RSI Değeri: 55.16
BINANCE:ZECUSDT için RSI düşeni kırdı! RSI Değeri: 51.75
BINANCEUS:ADXUSDT için RSI düşeni kırdı! RSI Değeri: 32.24
BINANCEUS:AUDIOUSDT için RSI düşeni kırdı! RSI Değeri: 33.77
BINANCEUS:DIAUSDT için RSI düşeni kırdı! RSI Değeri: 31.35
BINANCEUS:ORBSUS